# Data Exploration and Visualization of Cosmic Strings in CMB
This notebook explores the Cosmic Microwave Background (CMB) data to identify potential cosmic string candidates.

Initial exploration of the CMB data using Healpy and Matplotlib.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.constants.codata2014 import alpha
from scipy.constants import h, c, k

# Planck function (spectral radiance)
def planck(f, T):
    x = h * f / (k * T)
    return (2 * h * f**3 / c**2) / np.expm1(x)


# Frequency range (Hz)
frequencies = np.linspace(1e10, 1e12, 1000)  # 10 GHz to 1000 GHz
temperature = 2.725  # CMB temperature in Kelvin

# Compute spectral radiance
radiance = planck(frequencies, temperature)

# Plot spectrum
plt.figure(figsize=(10, 6))
plt.plot(frequencies / 1e9, radiance, color='darkblue', label=f'T = {temperature} K')
plt.title('CMB Blackbody Spectrum at T = 2.725 K')
plt.xlabel('Frequency (GHz)')
plt.ylabel('Spectral Radiance B(ν) [W·sr⁻¹·m⁻²·Hz⁻¹]')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import healpy as hp
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage

# Load a Planck CMB temperature map
cmb_map = hp.read_map('data/COM_CMB_IQU-smica_2048_R3.00_full.fits', field=0)

# Convert to 2D image
nside = hp.get_nside(cmb_map)
npix = hp.nside2npix(nside)
# Convert the CMB map to a 2D image for visualization
img = hp.mollview(cmb_map, return_projected_map=True, nest=False, title='CMB Temperature Map', cmap='inferno', xsize=2000)




In [ ]:
# Apply edge detection (e.g., Sobel)
edges = ndimage.sobel(img)
# Plot the edges detected in the CMB map
plt.figure(figsize=(10, 5))
plt.imshow(edges, cmap='inferno')
plt.title('Possible Cosmic String Candidates (Edges in CMB)')
plt.colorbar()
plt.show()


In [ ]:
frequencies = np.linspace(10e9, 1000e9, 1000)  # 10 GHz to 1000 GHz

T_mean = 2.725
delta_T = 100e-6  # 100 µK typical fluctuation

plt.figure(figsize=(10, 6))
plt.plot(frequencies / 1e9, planck(frequencies, T_mean), label="T = 2.725 K", color='blue')
plt.plot(frequencies / 1e9, planck(frequencies, T_mean + delta_T), '--', label="+100 µK", color='green', alpha=0.8)
plt.plot(frequencies / 1e9, planck(frequencies, T_mean - delta_T), '--', label="-100 µK", color='red', alpha=0.8)
plt.xlabel("Frequency (GHz)")
plt.ylabel("Spectral Radiance (W·sr⁻¹·m⁻²·Hz⁻¹)")
plt.title("CMB Spectrum with Fluctuation Bounds (±100 µK)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Show the CMB spectrum with +100 µK fluctuations
plt.figure(figsize=(10, 6))
plt.plot(frequencies / 1e9, planck(frequencies, T_mean), label="T = 2.725 K", color='blue')
plt.plot(frequencies / 1e9, planck(frequencies, T_mean + delta_T), '--', label="+100 µK", color='green', alpha=0.8)
plt.xlabel("Frequency (GHz)")
plt.ylabel("Spectral Radiance (W·sr⁻¹·m⁻²·Hz⁻¹)")
plt.title("CMB Spectrum with Fluctuation Bounds (±100 µK)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Show the CMB spectrum with -100 µK fluctuations
plt.figure(figsize=(10, 6))
plt.plot(frequencies / 1e9, planck(frequencies, T_mean), label="T = 2.725 K", color='blue')
plt.plot(frequencies / 1e9, planck(frequencies, T_mean - delta_T), '--', label="-100 µK", color='red', alpha=0.8)
plt.xlabel("Frequency (GHz)")
plt.ylabel("Spectral Radiance (W·sr⁻¹·m⁻²·Hz⁻¹)")
plt.title("CMB Spectrum with Fluctuation Bounds (±100 µK)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
cl = hp.anafast(cmb_map)
plt.plot(cl[:200])
plt.title("Low-ℓ Multipoles – Texture Signature Region")
plt.xlabel("Multipole ℓ [ℓ: inverse angular scale]")
plt.ylabel("C_ℓ [Power: variance of temperature fluctuations]")
plt.grid()
plt.show()


In [ ]:
# Calculate the power spectrum
# Use Healpy to compute the power spectrum from the CMB map
cl = hp.anafast(cmb_map)
# Define multipole indices (ℓ)
# Create an array of multipole indices (ℓ)
ell = np.arange(len(cl))

# Plot the power spectrum with log scale to better visualize features
plt.figure(figsize=(10, 6))
# Plot the CMB power spectrum using a logarithmic scale
# Use ell[1:1500] to skip the monopole (ℓ=0)
plt.loglog(ell[1:1500], cl[1:1500], label='CMB Power Spectrum')

# Add theoretical curve for comparison (ℓ(ℓ+1)Cℓ ~ constant for scale-invariant spectrum)
# Scale the theoretical curve to match the actual CMB power spectrum (around 1e-10)
# Get a reference value from the actual power spectrum
ref_value = np.mean(cl[5:15])  # Use multipoles 5-15 as reference
# Create a properly scaled theoretical curve
# Theoretical curve for scale-invariant spectrum
theory_curve = ref_value * (ell[5] * (ell[5] + 1)) / (ell[1:1500] * (ell[1:1500] + 1))
# Plot the theoretical curve
plt.loglog(ell[1:1500], theory_curve, 'r--', label='Scale-invariant spectrum (∝ 1/ℓ(ℓ+1))')

# Highlight acoustic peaks region
# Overlay the acoustic peaks region (ℓ = 50 to 1500)
# This region is where the acoustic oscillations are most prominent
# Using a semi-transparent green band to indicate this region
plt.axvspan(50, 1500, color='green', alpha=0.1, label='Acoustic Peaks Region')

# Add labels and formatting
plt.title("CMB Power Spectrum (Log Scale)")
plt.xlabel("Multipole ℓ [ℓ: inverse angular scale]")
plt.ylabel("C_ℓ [Power: variance of temperature fluctuations]")
plt.legend()
plt.grid(True, which="both", ls="-", alpha=0.2)
plt.show()

In [ ]:
# Plot the power spectrum (ℓ = 0 to 199)
ell = np.arange(len(cl))  # Define ell as the array of multipole indices
plt.figure(figsize=(9, 5))
plt.plot(ell[:200], cl[:200], label='CMB Power')

# Overlay topological defect regions
plt.axvspan(2, 10, color='blue', alpha=0.3, label='Textures (ℓ ≈ 2–10)')
plt.axvspan(2, 5, color='red', alpha=0.3, label='Domain Walls (ℓ ≈ 2–5)')
plt.axvspan(50, 200, color='green', alpha=0.2, label='Cosmic Strings (ℓ ≈ 50–200)')
plt.axvspan(2, 3, color='orange', alpha=0.4, label='Monopoles (ℓ ≈ 2–3)')

# Labels and grid
plt.title("CMB Multipole Spectrum with Topological Defect Regions")
plt.xlabel("Multipole ℓ [ℓ: inverse angular scale]")
plt.ylabel("C_ℓ [Power: variance of temperature fluctuations]")
plt.grid()
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Plot the power spectrum (ℓ = 0 to 199)
ell = np.arange(len(cl))  # Define ell as the array of multipole indices
plt.figure(figsize=(9, 5))
plt.plot(ell[:50], cl[:50], label='CMB Power')

# Overlay topological defect regions
plt.axvspan(2, 10, color='blue', alpha=0.3, label='Textures (ℓ ≈ 2–10)')
plt.axvspan(2, 5, color='red', alpha=0.3, label='Domain Walls (ℓ ≈ 2–5)')
plt.axvspan(2, 3, color='orange', alpha=0.4, label='Monopoles (ℓ ≈ 2–3)')

# Labels and grid
plt.title("CMB Multipole Spectrum with Topological Defect Regions")
plt.xlabel("Multipole ℓ [ℓ: inverse angular scale]")
plt.ylabel("C_ℓ [Power: variance of temperature fluctuations]")
plt.grid()
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Use same `cl` from previous block
ell = np.arange(1, 200)  # Skip ℓ = 0 to avoid divide-by-zero
theta_deg = 180 / ell    # Angular scale in degrees

# Trim cl accordingly
cl_trim = cl[1:200]

# Plot C_ell vs angular scale θ
plt.plot(theta_deg, cl_trim)
plt.title("CMB Power Spectrum vs Angular Scale")
plt.xlabel("Angular Scale θ [degrees]")
plt.ylabel("C_ℓ [Power: variance of temperature fluctuations]")
plt.grid()
plt.gca().invert_xaxis()  # Larger angular scales on the left
plt.show()

In [ ]:
# ℓ = 1 to 199 (skip ℓ = 0 to avoid division by zero)
theta_deg = 180 / ell
cl_trim = cl[1:200]

# Plot power vs angular scale
plt.figure(figsize=(9, 5))
plt.plot(theta_deg, cl_trim, label='CMB Power')

# Overlay topological defect regions in degrees
plt.axvspan(18, 90, color='blue', alpha=0.3, label='Textures (θ ≈ 18° to 90°)')
plt.axvspan(36, 90, color='red', alpha=0.3, label='Domain Walls (θ ≈ 36° to 90°)')
plt.axvspan(1, 4, color='green', alpha=0.2, label='Cosmic Strings (θ ≈ 1° to 4°)')
plt.axvspan(60, 90, color='orange', alpha=0.4, label='Monopoles (θ ≈ 60° to 90°)')

# Labels and formatting
plt.title("CMB Power vs Angular Scale with Topological Defect Regions")
plt.xlabel("Angular Scale θ [degrees]")
plt.ylabel("C_ℓ [Power: variance of temperature fluctuations]")
plt.grid()
plt.gca().invert_xaxis()  # Large scales (low ℓ) on left
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
from skimage import draw  # Changed import to use skimage.draw instead of matplotlib.pyplot.draw

def simulate_cosmic_strings(map_size, num_strings):
    map_data = np.zeros((map_size, map_size))
    for _ in range(num_strings):
        x = np.random.randint(0, map_size)
        y = np.random.randint(0, map_size)
        dx = np.random.randint(-map_size//4, map_size//4)
        dy = np.random.randint(-map_size//4, map_size//4)
        rr, cc = draw.line(x, y, x+dx, y+dy)  # Now using skimage.draw.line
        map_data[rr % map_size, cc % map_size] += np.random.choice([-1, 1]) * 100e-6  # ~100 µK jump
    return map_data

# Simulate and plot
map_size = 1000  # Reduced size for better visualization
num_strings = 10
cosmic_map = simulate_cosmic_strings(map_size, num_strings)

plt.figure(figsize=(10, 10))
plt.imshow(cosmic_map, cmap='RdBu_r')
plt.colorbar(label='Temperature fluctuation (K)')
plt.title('Simulated Cosmic Strings')
plt.show()
